In [1]:
import pylab as plt
import numpy as np
import swot_ssh_utils as swot
import swot_swell_utils as swell
from cartopy import crs as ccrs
from netCDF4 import Dataset
import xarray as xr
from scipy import ndimage

import sys as sys
import pickle
import glob as glob
import datetime


import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.pyplot import cm
import cmocean
import cmocean.cm as cmo

#PATH_with_functions = '/home/ardhuin/TOOLS/OPTOOLS/PYTHON/'
#sys.path.append(PATH_with_functions)
from  spectral_analysis_functions import *
plt.rcParams.update({'font.size': 18,'savefig.facecolor':'white'})

cNorm = mcolors.Normalize(vmin=0, vmax=2)
jet = plt.get_cmap('jet')
mpl.rcParams.update({'figure.figsize':[10,6],'axes.grid' : True,'font.size': 14,'savefig.facecolor':'white'})

In [8]:
# Note : other tests with different spectral resolution are in directories LOOP, LOOP2, LOOP3
# First track for Figure 4.
cycle='547';tracks='008';latrange=[-22,-8];Lrange=[500,1100];thrange=[200,250];thstep=10;Hrange=[0,2];

# track for Figure S7
#cycle='550';tracks='017';latrange=[-33,-5];Lrange=[400,900];thrange=[180,240];thstep=15;Hrange=[0,3];

file_list = sorted(glob.glob('ANIMATIONS/SWOT_swell_spectra_'+cycle+'_'+tracks+'_left_*.npz'));
#file_list = sorted(glob.glob('LOOP3/SWOT_swell*_'+tracks+'left_*.npz'));
nfiles=len(file_list)
file_lisl = sorted(glob.glob('ANIMATIONS/SWOT_swell_spectra_'+cycle+'_'+tracks+'_right_*.npz'));
nfilel=len(file_lisl)
tracktag='cycle '+cycle+', track '+tracks;
print('We have ',nfiles,nfilel,' files.')
bigarray=np.zeros((nfiles+nfilel,50))
indf=0
obpfilter='H2'
for file_list in [file_list,file_lisl]:
  for file_box in file_list:
    #Eta_WW3_obp_H2=[]
    #print('reading file ',file_box)
    data = np.load(file_box,allow_pickle=True)
    for k in data.keys():
        if (indf==0):
            print(k+' = data["'+k+'"]')
        exec(k+' = data["'+k+'"]')
    bigarray[indf,0]=timec
    bigarray[indf,1]=lonc
    bigarray[indf,2]=latc
    bigarray[indf,3]=timeww3
    bigarray[indf,3]=0
    
    
    if obpfilter=='H':
        Hsnew,Lmm1new,Lmnew,dmnew=swell.SWOTspec_to_HsLm(E_SWOT,kx2,ky2,amask,HH,trackangle)
    else:
        Hsnew,Lmm1new,Lmnew,dmnew=swell.SWOTspec_to_HsLm(E_SWOT,kx2,ky2,amask,HH**2,trackangle)
    bigarray[indf,10]=Hsnew
    bigarray[indf,20]=Lmm1new
    bigarray[indf,30]=dmnew
    b2mask=amask
# uses a slightly wider mask for the model to be sure to catch the full partition 
    b2mask=ndimage.binary_dilation((amask > 0.5).astype(int))
    #
    if obpfilter=='H':
        Hsnew,Lmm1new,Lmnew,dmnew=swell.SWOTspec_to_HsLm(E_WW3_obp_H,kx2,ky2,b2mask,HH,trackangle)
    else:
        Hsnew,Lmm1new,Lmnew,dmnew=swell.SWOTspec_to_HsLm(E_WW3_obp_H2,kx2,ky2,b2mask,HH**2,trackangle)
    bigarray[indf,11]=Hsnew
    bigarray[indf,21]=Lmm1new
    bigarray[indf,31]=dmnew
    indf=indf+1





gr=[0.,205./255.,0.]
fig,axs=plt.subplots(1,3,figsize=(12,5))
vecHs=np.linspace(1,12,23)
axs[0].scatter(bigarray[0:nfiles,11],bigarray[0:nfiles,2],c='black',linewidth=1,label='model left');
axs[0].scatter(bigarray[nfiles+1:-1,11],bigarray[nfiles+1:-1,2],c='blue',linewidth=1,label='model  right');
axs[0].scatter(bigarray[nfiles+1:-1,10],bigarray[nfiles+1:-1,2],c=gr,linewidth=1,label='SWOT  right');
axs[0].scatter(bigarray[0:nfiles,10],bigarray[0:nfiles,2],c='red',linewidth=1,label='SWOT left');
#ax1.plot(vecHs,vecHs**1*1/264,c='black',linewidth=3,linestyle='dashed',label=r'eq. (23), n=1, $s_0=1$~m.');
axs[0].set_xlabel(r'$H_{18}$ (m)', fontsize=18)
axs[0].set_ylabel('latitude (deg)', fontsize=18)
axs[0].set_title(tracktag, fontsize=18)
axs[0].set_xlim(Hrange);
#axs[0].set_xlim([0,5]);
axs[0].set_ylim(latrange)

axs[1].scatter(bigarray[0:nfiles,21],bigarray[0:nfiles,2],c='black',linewidth=1,label='model left');
axs[1].scatter(bigarray[nfiles+1:-1,21],bigarray[nfiles+1:-1,2],c='blue',linewidth=1,label='model  right');
axs[1].scatter(bigarray[nfiles+1:-1,20],bigarray[nfiles+1:-1,2],c=gr,linewidth=1,label='SWOT  right');
axs[1].scatter(bigarray[0:nfiles,20],bigarray[0:nfiles,2],c='red',linewidth=1,label='SWOT left');
axs[1].set_xlabel(r'$L_{18}$ (m)', fontsize=18)
format = '%Y-%m-%dT%H:%M:%S'
axs[1].set_title(str(timec)[0:16], fontsize=18)
axs[1].set_xlim(Lrange)
axs[1].set_xticks(np.arange(Lrange[0],Lrange[1],100))
#axs[1].set_xlim([450,650])
#axs[1].set_xticks(np.arange(450,650,50))
axs[1].set_ylim(latrange)
leg = axs[1].legend(loc='lower right', fontsize=14)

lightcmap = cmocean.tools.lighten(cmo.ice_r, 0.5)
#ax.pcolormesh(Z, cmap=lightcmap)
#axs[2].pcolormesh(7*(ds_glob.longitude-area[0])+190,ds_glob.latitude,np.sqrt(U**2+V**2),cmap=lightcmap,rasterized=True,shading='nearest',alpha=0.5,vmin = 0, vmax =1)
#axs[2].scatter(7*(xt[ind]-area[0])+190,yt[ind],c='r',marker='+',s=30,linewidth=2)

axs[2].scatter(bigarray[0:nfiles,31],bigarray[0:nfiles,2],c='black',linewidth=1,label='Lm WW3');
axs[2].scatter(bigarray[nfiles+1:-1,31],bigarray[nfiles+1:-1,2],c='blue',linewidth=1,label='WW3 left');
axs[2].scatter(bigarray[nfiles+1:-1,30],bigarray[nfiles+1:-1,2],c=gr,linewidth=1,label='SWOT left');
axs[2].scatter(bigarray[0:nfiles,30],bigarray[0:nfiles,2],c='red',linewidth=1,label='Lm SWOT');
axs[2].set_xlabel(r'$\theta_{18}$ (deg. nautical,from)', fontsize=18)
axs[2].set_xlim(thrange)
axs[2].set_xticks(np.arange(thrange[0],thrange[1],thstep))
axs[2].set_ylim(latrange)
#axs[2].set_title(tracktag)
if obpfilter=='H':
    fig.savefig('fulltrack_track8_H.pdf')
else:
    fig.savefig('fulltrack_track8_H2.pdf')
       

We have  57 55  files.
Xbox = data["Xbox"]
Ybox = data["Ybox"]
sshbox_detrend = data["sshbox_detrend"]
sig0box = data["sig0box"]
kx2 = data["kx2"]
ky2 = data["ky2"]
ssh_spec = data["ssh_spec"]
sig0_spec = data["sig0_spec"]
coh = data["coh"]
ang = data["ang"]
amask = data["amask"]
bmask = data["bmask"]
lonc = data["lonc"]
latc = data["latc"]
timec = data["timec"]
timeww3 = data["timeww3"]
indww3 = data["indww3"]
trackangle = data["trackangle"]
Eta_SWOT = data["Eta_SWOT"]
coh_SWOT = data["coh_SWOT"]
phase_SWOT = data["phase_SWOT"]
Eta_WW3_obp_H = data["Eta_WW3_obp_H"]
Eta_WW3_obp_H2 = data["Eta_WW3_obp_H2"]
HH = data["HH"]
Hs_SWOT_all = data["Hs_SWOT_all"]
Hs_SWOT = data["Hs_SWOT"]
Hs_WW3 = data["Hs_WW3"]
Hs_WW3_all = data["Hs_WW3_all"]
Hs_WW3_cut = data["Hs_WW3_cut"]
Hs_SWOT_mask = data["Hs_SWOT_mask"]
Hs_WW3_mask = data["Hs_WW3_mask"]
HsWW3 = data["HsWW3"]
Tm0m1WW3 = data["Tm0m1WW3"]
QkkWW3 = data["QkkWW3"]
Lm_SWOT = data["Lm_SWOT"]
Lm_WW3 = data["Lm_WW3"]
dm_SWOT = data["dm_SWOT"]
dm_

ValueError: operands could not be broadcast together with shapes (21,20) (42,40) 